# 🎬 Remoção Automática de Legendas de Vídeo com OCR + Inpainting

In [ ]:
!pip install -q opencv-python-headless pytesseract moviepy numpy!apt-get -qq install -y tesseract-ocr

In [ ]:
from google.colab import filesuploaded = files.upload()filename = next(iter(uploaded))

In [ ]:
import cv2import numpy as npimport pytesseractfrom moviepy.editor import VideoFileClip, ImageSequenceClipimport osfrom tqdm import tqdm

In [ ]:
def detect_subtitles(frame):    height = frame.shape[0]    roi = frame[int(height * 0.75):, :]    gray = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)    _, mask = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)    text = pytesseract.image_to_string(mask)    return text.strip() != '', mask

In [ ]:
def remove_subtitles(frame, mask):    height = frame.shape[0]    full_mask = np.zeros(frame.shape[:2], dtype=np.uint8)    full_mask[int(height * 0.75):, :] = mask    result = cv2.inpaint(frame, full_mask, 3, cv2.INPAINT_TELEA)    return result

In [ ]:
def process_video(input_path, output_path):    clip = VideoFileClip(input_path)    processed_frames = []    for frame in tqdm(clip.iter_frames()):        has_subtitles, mask = detect_subtitles(frame)        if has_subtitles:            frame = remove_subtitles(frame, mask)        processed_frames.append(frame)    final_clip = ImageSequenceClip(processed_frames, fps=clip.fps)    final_clip.write_videofile(output_path, codec='libx264')

In [ ]:
output_name = 'output_' + filenameprocess_video(filename, output_name)

In [ ]:
from google.colab import filesfiles.download(output_name)